In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

os.environ['CUDA_VISIBLE_DEVICES']='2, 3'

from scipy import misc
import sys
import os
import random
from tqdm import tqdm

import numpy as np

import cv2

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import facenet
import detect_face

In [ ]:
ROOT_DIR = os.getcwd()

DATA_PATH = os.path.join(ROOT_DIR, "data")

MODEL_PATH = os.path.join(ROOT_DIR, "model")

MTCNN_MODEL_PATH = os.path.join(MODEL_PATH, "mtcnn")

IMG_IN_PATH = os.path.join(DATA_PATH, "pic")

IMG_OUT_PATH = os.path.join(DATA_PATH, "pic_crops")

In [ ]:
with tf.Graph().as_default():
    config = tf.ConfigProto(allow_soft_placement=True)
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, MTCNN_MODEL_PATH)

In [ ]:
minsize = 20  # 最小的臉部的大小
threshold = [0.6, 0.7, 0.7]  # 三個網絡(P-Net, R-Net, O-Net)的閥值
factor = 0.709  # scale factor

margin = 44 # 在裁剪人臉時的邊框margin
image_size = 182 # 160 + 22

In [ ]:
random_key = np.random.randint(0, high=99999)
bounding_boxes_filename = os.path.join(IMG_OUT_PATH, 'bounding_boxes_%05d.txt' % random_key)

In [ ]:
cv2.namedWindow("frame",0)

cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret, img = cap.read()
    if ret == True:
        cv2.imshow("frame", img)
        k = cv2.waitKey(100)
        if k == ord("z") or k == ord("Z"):
            cv2.imwrite("2.jpg", img)
            break
cap.release()
cv2.destroyWindow("frame")